In [ ]:
from typing import NamedTuple

In [ ]:
class A(NamedTuple):
    a: int
    b: int

In [ ]:
c = A(1, 2)

In [ ]:
for i in c:
    print(i)

In [1]:
import gymnasium as gym

In [2]:
envs = gym.vector.make("CartPole-v1", num_envs=2)

In [3]:
envs.reset()

(array([[-0.01605514, -0.00961014, -0.0445478 ,  0.00539178],
        [-0.04985068,  0.04258873,  0.00790535, -0.03947148]],
       dtype=float32),
 {})

In [1]:
from pathlib import Path

In [2]:
Path().cwd()

WindowsPath('g:/My Drive/Development/Python Projects/MateX_ray/tests')